## Data.go.kr 에서 openAPI 키를 발급받아 화장품원료정보 데이터 수집

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://apis.data.go.kr/1471000/CsmtcsIngdCpntInfoService01/getCsmtcsIngdCpntInfoService01"
service_key = "qmfDs9T23gzvs0ps4giAyVbQdYEYDxWLfhh/T3LIOGbU1UXxgvj/gmKEMkYG9w4wh6JalDkgKgTgT8TVtnyrKA=="
num_of_rows = 100
page_no = 209
type_param = "xml"

params = {
        'serviceKey': service_key,
        'numOfRows': num_of_rows,
        'pageNo': page_no,
        'type': type_param,
    }

response = requests.get(url, params=params)

soup = BeautifulSoup(response.text, 'xml')

print(response.text)

<?xml version="1.0" encoding="UTF-8"?>
<response>
<header>
<resultCode>00</resultCode>
<resultMsg>NORMAL SERVICE.</resultMsg>
</header>
<body>
<numOfRows>100</numOfRows>
<pageNo>209</pageNo>
<totalCount>20836</totalCount>
<items>
<item>
<INGR_KOR_NAME>폴리글리세릴-10올리에이트/팔미테이트/스테아레이트</INGR_KOR_NAME>
<INGR_ENG_NAME>Polyglyceryl-10 Oleate/Palmitate/Stearate</INGR_ENG_NAME>
<CAS_NO>67884-82-1</CAS_NO>
<ORIGIN_MAJOR_KOR_NAME>이 원료는 올레익애씨드, 팔미틱애씨드 및 스테아릭애씨드 혼합물과 폴리글리세린-10의 에스터이다.</ORIGIN_MAJOR_KOR_NAME>
<INGR_SYNONYM/>
</item>
<item>
<INGR_KOR_NAME>키위잎추출물</INGR_KOR_NAME>
<INGR_ENG_NAME>Actinidia Chinensis (Kiwi) Leaf Extract</INGR_ENG_NAME>
<CAS_NO/>
<ORIGIN_MAJOR_KOR_NAME>이 원료는 키위 Actinidia chinensis의 잎에서 추출한 것이다.</ORIGIN_MAJOR_KOR_NAME>
<INGR_SYNONYM/>
</item>
<item>
<INGR_KOR_NAME>사르코탈리아 키르쿰킹타</INGR_KOR_NAME>
<INGR_ENG_NAME>Sarcothalia Circumcincta</INGR_ENG_NAME>
<CAS_NO/>
<ORIGIN_MAJOR_KOR_NAME>이 원료는 조류의 일종인 Sarcothalia circumcincta이다.</ORIGIN_MAJOR_KOR_NAME>
<INGR_SYNONYM/>
</item>
<item>
<

In [16]:
data = {'A': [1, 2, 3],
        'B': ['a', 'b', 'c'],
        'C': [10.5, 20.3, 30.1]}

df = pd.DataFrame(data)
df

,A,B,C
0,1,a,10.5
1,2,b,20.3
2,3,c,30.1


In [ ]:
''' 
num_of_rows를 전체 데이터의 개수로 설정하고 page_no를 1로 설정한다면, 한 번의 API 요청으로 전체 데이터를 한꺼번에 가져올 수 있지만 이렇게 한 번에 많은 양의 데이터를 가져오는 경우, 응답이 크기 때문에 처리 시간이 오래 걸릴 수 있고, API 제공자의 정책에 따라 요청에 대한 제한이 있다.

일반적으로 대용량의 데이터를 다룰 때는 페이지네이션(pagination)을 사용하여 여러 번에 걸쳐 데이터를 가져오는 방식을 선호함. 
페이지네이션을 사용하면 데이터를 조금씩 나눠서 가져올 수 있어서 응답 시간이 단축되고, 서버에 부담을 덜 줄 수 있음.
'''

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://apis.data.go.kr/1471000/CsmtcsIngdCpntInfoService01/getCsmtcsIngdCpntInfoService01"
service_key = "qmfDs9T23gzvs0ps4giAyVbQdYEYDxWLfhh/T3LIOGbU1UXxgvj/gmKEMkYG9w4wh6JalDkgKgTgT8TVtnyrKA=="
num_of_rows = 100
type_param = "xml"

all_data = {'INGR_KOR_NAME': [], 'INGR_ENG_NAME': [], 'CAS_NO': [], 'ORIGIN_MAJOR_KOR_NAME': [], 'INGR_SYNONYM': []}

total_count = 20836  # 전체 데이터 개수
for page_no in range(1, (total_count // num_of_rows) + 2):  # 페이지 수 계산 (20836/100 = 208페이지 -> 20800개이므로 36개가 남아 209페이지까지 돌려야함)
    params = {
        'serviceKey': service_key,
        'pageNo': page_no,
        'numOfRows': num_of_rows,
        'type': type_param,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'xml')

        for item in soup.find_all('item'):
            inger_kor_name_element = item.find('INGR_KOR_NAME')
            inger_eng_name_element = item.find('INGR_ENG_NAME')
            cas_no_element = item.find('CAS_NO')
            origin_major_kor_name_element = item.find('ORIGIN_MAJOR_KOR_NAME')
            ingr_synonym_element = item.find('INGR_SYNONYM')

            all_data['INGR_KOR_NAME'].append(inger_kor_name_element.text)
            all_data['INGR_ENG_NAME'].append(inger_eng_name_element.text)
            all_data['CAS_NO'].append(cas_no_element.text)
            all_data['ORIGIN_MAJOR_KOR_NAME'].append(origin_major_kor_name_element.text)
            all_data['INGR_SYNONYM'].append(ingr_synonym_element.text)

    else:
        print(f"Error: {response.status_code}")
        print(response.text)

df_all = pd.DataFrame(all_data)
df_all.index = range(1, len(df_all) + 1)

df_all

,INGR_KOR_NAME,INGR_ENG_NAME,CAS_NO,ORIGIN_MAJOR_KOR_NAME,INGR_SYNONYM
1,가공소금,,,,
2,가지열매추출물,Solanum Melongena (Eggplant) Fruit Extract,84012-19-1,이 원료는 가지(Eggplant) Solanum melongena의 열매에서 추출한...,가지추출물
3,구멍쇠미역추출물,Agarum Cribrosum Extract,,이 원료는 조류의 일종인 구멍쇠미역 Agarum cribosum에서 추출한 것이다.,
4,루핀아미노산,Lupine Amino Acids,,이 원료는 루핀 단백질의 완전 가수분해로 얻은 아미노산의 혼합물이다.,
5,류신,Leucine,"328-39-2(DL-)\r,61-90-5(L-)",이 원료는 다음의 구조를 갖는 아미노산이다.,
...,...,...,...,...,...
20832,오이열매소포,,,이 원료는 오이 Cucumis sativus의 열매에서 분리한 소포이다.,
20833,순비기나무꽃,,,이 원료는 순비기나무 Vitex rotundifolia의 꽃을 건조한 것이다.,
20834,에이치이에이아이피디아이아이소시아누레이트트라이머/폴리카프로락톤다이올코폴리머,HEA IPDI Isocyanurate Trimer/Polycaprolactone ...,71243-87-3,이 원료는 엡실론-카프로락톤을 다이에틸렌글라이콜로 축합하고 2-하이드록시에틸아크릴레...,
20835,석류씨소포,,,이 원료는 석류나무 Punica granatum의 씨에서 분리한 소포이다.,


In [11]:
df_all.to_csv('./data/02.데이터수집_01크롤링_화장품국문영문_화장품원료.csv', encoding='utf-8-sig', index=False)

In [12]:
df_all = pd.read_csv('./data/02.데이터수집_01크롤링_화장품국문영문_화장품원료.csv', encoding='utf-8')
df_all

,INGR_KOR_NAME,INGR_ENG_NAME,CAS_NO,ORIGIN_MAJOR_KOR_NAME,INGR_SYNONYM
0,가공소금,NaN,NaN,NaN,NaN
1,가지열매추출물,Solanum Melongena (Eggplant) Fruit Extract,84012-19-1,이 원료는 가지(Eggplant) Solanum melongena의 열매에서 추출한...,가지추출물
2,구멍쇠미역추출물,Agarum Cribrosum Extract,NaN,이 원료는 조류의 일종인 구멍쇠미역 Agarum cribosum에서 추출한 것이다.,NaN
3,루핀아미노산,Lupine Amino Acids,NaN,이 원료는 루핀 단백질의 완전 가수분해로 얻은 아미노산의 혼합물이다.,NaN
4,류신,Leucine,"328-39-2(DL-)\r,61-90-5(L-)",이 원료는 다음의 구조를 갖는 아미노산이다.,NaN
...,...,...,...,...,...
20831,오이열매소포,NaN,NaN,이 원료는 오이 Cucumis sativus의 열매에서 분리한 소포이다.,NaN
20832,순비기나무꽃,NaN,NaN,이 원료는 순비기나무 Vitex rotundifolia의 꽃을 건조한 것이다.,NaN
20833,에이치이에이아이피디아이아이소시아누레이트트라이머/폴리카프로락톤다이올코폴리머,HEA IPDI Isocyanurate Trimer/Polycaprolactone ...,71243-87-3,이 원료는 엡실론-카프로락톤을 다이에틸렌글라이콜로 축합하고 2-하이드록시에틸아크릴레...,NaN
20834,석류씨소포,NaN,NaN,이 원료는 석류나무 Punica granatum의 씨에서 분리한 소포이다.,NaN
